In [1]:
import numpy as np
import cv2
import tensorflow as tf
import time

import configparser
config = configparser.ConfigParser()
config.read("config.ini")

# from test import *
# from models.utilities import *
from models.subclasses import *
from models.predict import *

In [2]:
# Static values
PATH = 'testVideos/IMG_9367.MOV'

test_image, _ = load_image("testVideos/35506150_cbdb630f4f.jpg")

test_image2, _ = load_image("testVideos/IMG_3004.jpg")

#how many frame to play until pause
SHOW_FRAME = 60

units = int(config['config']['units'])
embedding_dim = int(config['config']['embedding_dim'])

In [3]:
img_name_train, cap_train, img_name_val, cap_val, vocabulary, tokens_shape = load_dataset()
word_to_index, index_to_word = index_vocab(vocabulary)

In [12]:
temp = tf.convert_to_tensor(
    word_to_index
)
temp('<start>')

ValueError: Attempt to convert a value (<keras.layers.preprocessing.string_lookup.StringLookup object at 0x000001B00FCDE700>) with an unsupported type (<class 'keras.layers.preprocessing.string_lookup.StringLookup'>) to a Tensor.

In [4]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, len(vocabulary))

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [5]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
ckpt.restore(ckpt_manager.latest_checkpoint)

In [6]:
_, _, image_features_extract_model = load_models()

In [7]:
def predict_all(frame):
    img = tf.keras.layers.Resizing(224, 224)(frame)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    result = predict_image(img, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)
    temp = Image.fromarray(frame, 'RGB')
#     temp.show()
    print(result)
    return result

In [8]:
predict_image(test_image, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)

['a',
 'man',
 'lounging',
 'on',
 'one',
 'of',
 'greenery',
 'and',
 'about',
 'to',
 'relax',
 'on',
 'a',
 'seat',
 '<end>']

In [11]:
predict_image(test_image2, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)

['a',
 'grassy',
 'field',
 'with',
 'many',
 'purple',
 'dots',
 'on',
 'top',
 'of',
 'a',
 'big',
 'green',
 'field.',
 '<end>']

## Run the app

In [10]:
#loop over all frames  'space' = next frame | 'q' = quit
cap = cv2.VideoCapture(PATH)
if (cap.isOpened()== False):
    print("Error opening video stream or file")
else:
    print("Statred capturing")

total_frames = 0
while cap.isOpened():
    HasFrames, frame = cap.read()
    #if vidoe is not done do
    if HasFrames:
        total_frames += 1
        cv2.imshow('Video', frame)
        #when you reach the pause frame do
        if((total_frames % SHOW_FRAME) == 0):
            #Caption 'frame'
            start = time.time()
            # print(frame.shape)
            predict_all(frame)
            print(f'Time taken for 1 image {time.time()-start:.4f} sec\n')
            #press 'E' to get next frame
            if(cv2.waitKey(5000) == ord('e')):
                continue

        if(cv2.waitKey(25) == ord('q')):
            break    

    else:
        break

cap.release()
cv2.destroyAllWindows()

Statred capturing
['a', 'picture', 'of', 'a', 'restaurant', 'is', 'blue', 'lamps', 'in', 'the', 'rain', '<end>']
Time taken for 1 image 0.6970 sec

['a', 'potted', 'plant', 'in', 'front', 'of', 'pedestal', 'in', 'a', 'sink.', '<end>']
Time taken for 1 image 0.6160 sec

['a', 'green', 'bench', 'in', 'a', 'courtyard', 'looking', 'hat', 'sitting', 'gym', '<end>']
Time taken for 1 image 0.6220 sec

